In [1]:
import syft as sy

In [84]:
duet = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [85]:
duet2 = sy.join_duet(loopback=True)


🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [86]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: a125f64141e74b27a8d8c26e60b363ca>,[Dataset],split 1,<class 'torch.Tensor'>


In [88]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 1dd6e018f9be497ea0a9bdeb131d37fe>,[Dataset],split 2,<class 'torch.Tensor'>


on va créer le mm model pour les 2 data ouwners car il ont le mm type de donnée 

In [89]:
class SyNet(sy.Module): 
    def __init__(self,torch_ref): 
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.lin = self.torch_ref.nn.Linear(1,1)
        
    def forward(self,x): 
        x = self.lin(x)
        return x

In [90]:
import torch 


In [91]:
remote_torch_1 = duet.torch
remote_torch_2 = duet2.torch

In [92]:
model1 = SyNet(torch)
model1_ptr = model1.send(duet)
params1 = model1.parameters()
optim1 = remote_torch_1.optim.SGD(params=params1,lr=0.1)

In [93]:
model2 = SyNet(torch)
model2_ptr = model2.send(duet2)
params2 = model2.parameters()
optim2 = remote_torch_2.optim.SGD(params=params2,lr=0.1)

In [100]:
data_pointer1 = duet.store[0]
data_pointer2= duet2.store[0]

In [101]:
duet2.store[0]

In [102]:
model3 = torch.nn.Linear(2,1)

optim3 = torch.optim.SGD(params=model3.parameters(),lr=0.1)

target = torch.tensor([
    [0.0],
    [1.0], 
    [0.0], 
    [1.0]
])

In [103]:
for iter in range(20): 
    optim1.zero_grad()
    optim2.zero_grad()
    optim3.zero_grad()
    
    activation_ptr1 = model1(data_pointer1)
    activation1 = activation_ptr1.clone().get(request_block=True)
    
    activation_ptr2 = model2(data_pointer2)
    activation2 = activation_ptr2.clone().get(request_block=True)
    
    
    #mtn on va faire une concatenation des  2 resultat des remote models puis on va les donner a notre model local (model3)
    
    #on a fait une concatenation car les donnée sont séparer entre plusieurs data ouwners
    activation = torch.cat((activation1,activation2),1)
    
    #le model3 il va nous donner des resultat suivant la concatenation des 2 resultat des remote models
    pred = model3(activation)
    
    loss= ((pred - target)**2).sum()
    loss.backward()
    
    #mtn on va avoir les gradients des 2 activations (2 models) pour pouvoir les envoyer a travers l'activation pointer
    # on envois les gradient a travers le duet server
    
    grad1_ptr = activation1.grad.clone().send(duet1)
    grad2_ptr = activation2.grad.clone().send(duet2)
    
    activation_ptr1.backward(grad1_ptr)
    activation_ptr2.backward(grad2_ptr)
    
    
    optim1.step()
    optim2.step()
    optim3.step()
    
    print(loss)
    

TypeError: linear(): argument 'input' (position 1) must be Tensor, not TensorPointer

In [59]:
type(data_pointer2)

syft.proxy.torch.TensorPointer

In [108]:
data_pointer1.get(reason="testinggg")


[2021-08-11T05:04:24.692156+0100][CRITICAL][logger]][15012] You do not have permission to .get() Object with ID: <UID: a125f64141e74b27a8d8c26e60b363ca>Please submit a request.


AuthorizationException: You do not have permission to .get() Object with ID: <UID: a125f64141e74b27a8d8c26e60b363ca>Please submit a request.